In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
X=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
test=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [5]:
X.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [6]:
test.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
X1=X["full_text"]
t1=test["full_text"]

In [9]:
X1

0       I think that students would benefit from learn...
1       When a problem is a change you have to let it ...
2       Dear, Principal\n\nIf u change the school poli...
3       The best time in life is when you become yours...
4       Small act of kindness can impact in other peop...
                              ...                        
3906    I believe using cellphones in class for educat...
3907    Working alone, students do not have to argue w...
3908    "A problem is a chance for you to do your best...
3909    Many people disagree with Albert Schweitzer's ...
3910    Do you think that failure is the main thing fo...
Name: full_text, Length: 3911, dtype: object

In [10]:
t1

0    when a person has no experience on a job their...
1    Do you think students would benefit from being...
2    Thomas Jefferson once states that "it is wonde...
Name: full_text, dtype: object

In [11]:
X_new=X1.append(t1,ignore_index=True)

In [12]:
tfidf = TfidfVectorizer()
result = tfidf.fit_transform(X_new)

In [13]:
tfidf=result.toarray()

In [14]:
tfidf.shape

(3914, 21370)

In [15]:
test=tfidf[len(X1):]
X2=tfidf[:len(X1)]

In [16]:
test.shape

(3, 21370)

In [17]:
X.shape

(3911, 8)

In [18]:
X2.shape

(3911, 21370)

In [19]:
IP=X2.shape[1]

In [20]:
IP

21370

In [21]:
X1_train = X.loc[:int(len(X1.index)*0.8)]
y_train = X1_train[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]
X1_val = X.loc[int(len(X1.index)*0.8)+1:int(len(X1.index)*0.9)]
y_val = X1_val[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]
X1_test = X.loc[int(len(X1.index)*0.9)+1:]
y_test = X1_test[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]

In [22]:
tfidf_train=X2[:int(len(X.index)*0.8)+1]
tfidf_val=X2[int(len(X.index)*0.8)+1:int(len(X.index)*0.9)+1]
tfidf_test=X2[int(len(X.index)*0.9)+1:]

In [23]:
print(tfidf_train.shape)
print(tfidf_test.shape)
print(tfidf_val.shape)

(3129, 21370)
(391, 21370)
(391, 21370)


In [24]:
y1_train=y_train["syntax"]
y2_train=y_train["cohesion"]
y3_train=y_train["vocabulary"]
y4_train=y_train["phraseology"]
y5_train=y_train["grammar"]
y6_train=y_train["conventions"]

y1_val=y_val["syntax"]
y2_val=y_val["cohesion"]
y3_val=y_val["vocabulary"]
y4_val=y_val["phraseology"]
y5_val=y_val["grammar"]
y6_val=y_val["conventions"]

y1_test=y_test["syntax"]
y2_test=y_test["cohesion"]
y3_test=y_test["vocabulary"]
y4_test=y_test["phraseology"]
y5_test=y_test["grammar"]
y6_test=y_test["conventions"]

In [25]:
print(tfidf_train.shape)
print(y1_train.shape)
print(tfidf_val.shape)
print(y1_val.shape)
print(tfidf_test.shape)
print(y1_test.shape)

(3129, 21370)
(3129,)
(391, 21370)
(391,)
(391, 21370)
(391,)


In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K 
from sklearn.metrics import mean_squared_error

In [27]:
print(tfidf_train.shape)
print(tfidf_val.shape)
print(tfidf_test.shape)

(3129, 21370)
(391, 21370)
(391, 21370)


In [28]:
lstm_embed_train = []
for i in range(0,len(tfidf_train)):
    lstm_embed_train.append(np.asarray([tfidf_train[i]]))

lstm_embed_test = []
for i in range(0,len(tfidf_test)):
    lstm_embed_test.append(np.asarray([tfidf_test[i]]))

lstm_embed_val = []
for i in range(0,len(tfidf_val)):
    lstm_embed_val.append(np.asarray([tfidf_val[i]]))

lstm_embed_test = np.asarray(lstm_embed_test)
lstm_embed_train = np.asarray(lstm_embed_train)
lstm_embed_val = np.asarray(lstm_embed_val)
print(lstm_embed_train.shape)
print(lstm_embed_val.shape)
print(lstm_embed_test.shape)

(3129, 1, 21370)
(391, 1, 21370)
(391, 1, 21370)


In [29]:
print(tfidf_train.shape)
print(tfidf_val.shape)
print(tfidf_test.shape)

(3129, 21370)
(391, 21370)
(391, 21370)


In [30]:
lstm_embed_train_final = []
for i in range(0,len(test)):
    lstm_embed_train_final.append(np.asarray([test[i]]))
lstm_embed_test_final = np.asarray(lstm_embed_train_final)
lstm_embed_test_final.shape

(3, 1, 21370)

In [31]:
def get_model_lstm():
    """Define the model."""
    
    model = Sequential()
    model.add(LSTM(128, input_shape=[1,IP], activation = 'tanh',return_sequences=True))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='relu'))

    model.summary()
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [32]:
lstm_model1 = get_model_lstm()
lstm_model1.fit(lstm_embed_train, y1_train, epochs=100,validation_data=(lstm_embed_val,y1_val))
lstm_result_1 = lstm_model1.predict(lstm_embed_test_final).flatten()
lstm_result_1 = [int(r) for r in lstm_result_1]
len(lstm_result_1)

2022-11-24 18:54:00.614113: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            11007488  
_________________________________________________________________
dropout (Dropout)            (None, 1, 128)            0         
_________________________________________________________________
dense (Dense)                (None, 1, 32)             4128      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              33        
Total params: 11,011,649
Trainable params: 11,011,649
Non-trainable params: 0
_________________________________________________________________


2022-11-24 18:54:01.482198: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
98/98 [==============================] - 9s 66ms/step - loss: 1.4689 - accuracy: 0.0029 - val_loss: 0.3982 - val_accuracy: 0.0026
Epoch 2/100
98/98 [==============================] - 6s 61ms/step - loss: 0.5370 - accuracy: 0.0029 - val_loss: 0.3468 - val_accuracy: 0.0026
Epoch 3/100
98/98 [==============================] - 6s 61ms/step - loss: 0.4737 - accuracy: 0.0029 - val_loss: 0.3293 - val_accuracy: 0.0026
Epoch 4/100
98/98 [==============================] - 6s 61ms/step - loss: 0.4292 - accuracy: 0.0029 - val_loss: 0.3180 - val_accuracy: 0.0026
Epoch 5/100
98/98 [==============================] - 6s 60ms/step - loss: 0.3893 - accuracy: 0.0029 - val_loss: 0.3212 - val_accuracy: 0.0026
Epoch 6/100
98/98 [==============================] - 6s 59ms/step - loss: 0.3465 - accuracy: 0.0029 - val_loss: 0.3262 - val_accuracy: 0.0026
Epoch 7/100
98/98 [==============================] - 6s 60ms/step - loss: 0.3231 - accuracy: 0.0029 - val_loss: 0.3337 - val_accuracy: 0.0026
Epoch 

3

In [33]:
lstm_model2 = get_model_lstm()
lstm_model2.fit(lstm_embed_train, y2_train, epochs=100,validation_data=(lstm_embed_val,y2_val))
lstm_result_2 = lstm_model2.predict(lstm_embed_test_final).flatten()
lstm_result_2 = [int(r) for r in lstm_result_2]
len(lstm_result_2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 128)            11007488  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 32)             4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1)              33        
Total params: 11,011,649
Trainable params: 11,011,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
98/98 [==============================] - 9s 66ms/step - loss: 1.5516 - accuracy: 0.0019 - val_lo

3

In [34]:
lstm_model3 = get_model_lstm()
lstm_model3.fit(lstm_embed_train, y3_train, epochs=100,validation_data=(lstm_embed_val,y3_val))
lstm_result_3 = lstm_model3.predict(lstm_embed_test_final).flatten()
lstm_result_3 = [int(r) for r in lstm_result_3]
len(lstm_result_3)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 128)            11007488  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1, 32)             4128      
_________________________________________________________________
dropout_5 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1, 1)              33        
Total params: 11,011,649
Trainable params: 11,011,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
98/98 [==============================] - 9s 66ms/step - loss: 1.5184 - accuracy: 6.3918e-04 - va

3

In [35]:
lstm_model4 = get_model_lstm()
lstm_model4.fit(lstm_embed_train, y4_train, epochs=100,validation_data=(lstm_embed_val,y4_val))
lstm_result_4 = lstm_model4.predict(lstm_embed_test_final).flatten()
lstm_result_4 = [int(r) for r in lstm_result_4]
len(lstm_result_4)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 128)            11007488  
_________________________________________________________________
dropout_6 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 1, 32)             4128      
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1, 1)              33        
Total params: 11,011,649
Trainable params: 11,011,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
98/98 [==============================] - 9s 66ms/step - loss: 1.5475 - accuracy: 0.0029 - val_lo

3

In [36]:
lstm_model5 = get_model_lstm()
lstm_model5.fit(lstm_embed_train, y5_train, epochs=100,validation_data=(lstm_embed_val,y5_val))
lstm_result_5 = lstm_model5.predict(lstm_embed_test_final).flatten()
lstm_result_5 = [int(r) for r in lstm_result_5]
len(lstm_result_5)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 128)            11007488  
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 1, 32)             4128      
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 1, 1)              33        
Total params: 11,011,649
Trainable params: 11,011,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
98/98 [==============================] - 9s 66ms/step - loss: 1.5716 - accuracy: 0.0019 - val_lo

3

In [37]:
lstm_model6 = get_model_lstm()
lstm_model6.fit(lstm_embed_train, y6_train, epochs=100,validation_data=(lstm_embed_val,y6_val))
lstm_result_6 = lstm_model6.predict(lstm_embed_test_final).flatten()
lstm_result_6 = [int(r) for r in lstm_result_6]
len(lstm_result_6)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1, 128)            11007488  
_________________________________________________________________
dropout_10 (Dropout)         (None, 1, 128)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 1, 32)             4128      
_________________________________________________________________
dropout_11 (Dropout)         (None, 1, 32)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1, 1)              33        
Total params: 11,011,649
Trainable params: 11,011,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
98/98 [==============================] - 9s 65ms/step - loss: 1.6264 - accuracy: 0.0038 - val_lo

3

In [38]:
test.shape

(3, 21370)

In [39]:
t1=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [40]:
final=pd.DataFrame()

final['text_id']=t1["text_id"]
final['syntax']=lstm_result_1
final['cohesion']=lstm_result_2
final['vocabulary']=lstm_result_3
final['phraseology']=lstm_result_4
final['grammar']=lstm_result_5
final['conventions']=lstm_result_6

In [41]:
final

,text_id,syntax,cohesion,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2,2,2,2,2,2
1,000BAD50D026,2,2,2,2,2,3
2,00367BB2546B,2,2,3,2,3,2


In [42]:
final.to_csv('submission.csv', index=False)

In [43]:
final

,text_id,syntax,cohesion,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2,2,2,2,2,2
1,000BAD50D026,2,2,2,2,2,3
2,00367BB2546B,2,2,3,2,3,2
